<a href="https://colab.research.google.com/github/omar-sharif03/NAACL-SRW-2021/blob/main/ML_Algorithms_NAACL_(Emotion_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras import models
from keras import layers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)

CPU times: user 1.64 s, sys: 317 ms, total: 1.96 s
Wall time: 2.63 s


## Reading cleaned data

In [ ]:
path = '/content/drive/MyDrive/Projects/NAACL-EmotionClassification/'

raw_train_data = pd.read_excel(path+'train.xlsx') 
raw_val_data = pd.read_excel(path + 'val.xlsx')
raw_test_data = pd.read_excel(path + 'test.xlsx')

In [ ]:
raw_train_data.head(10)

,Unnamed: 0,cleaned,classes,labels
0,0,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,sadness,1
1,1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,sadness,1
2,2,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,disgust,3
3,3,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,sadness,1
4,4,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,fear,5
5,5,কান্নার জল সবাই দেখে হৃদয়ের কষ্ট কেউ দেখেনা প...,sadness,1
6,6,নির্বাচনের সময় কস্ট করে ভোট চাইসে এর চেয়ে বেশি...,disgust,3
7,7,দাদা আপনার video গুলো ভালো লাগে ডকুমেন্টারি টা...,joy,0
8,8,বাস্তবতা এটাই কেউ সারাজীবন সুখ চেয়েও পায় নাআর ...,sadness,1
9,9,কিন্ত ভয় কিসের থেকে আমি জানতামনা কিন্ত আমার মন...,fear,5


In [ ]:
raw_val_data.head(10)

,Unnamed: 0,cleaned,classes,labels
0,0,তোরা সবাই বিটিভির পর্দায় ফিরে আয়🙂তোদের জন্য মু...,disgust,3
1,1,২০১৭ সালে বৈশ্বিকভাবে সেপসিসে ৪ কোটি ৮৯ লাখ মা...,sadness,1
2,2,আপনার দুই গালে দুইটা কষে থাপ্পড় দেয়া যাইতো চিন...,anger,4
3,3,একজন হিন্দু হয়ে আজকে একটু ভুলের জন্য বীফ কারি ...,sadness,1
4,4,আজ দুঃখ ভোলার দিন আজ মন হবে যে রঙিন আজ প্রাণ খ...,joy,0
5,5,তার থেকেও বেশী কষ্ট হচ্ছে তাঁর দাফনের দৃশ্য দে...,sadness,1
6,6,ভাই তোমার ভিডিও দেখি ওই হিজরার উপরে আমার মাথা ...,anger,4
7,7,কত দিন পর বারো বছর এ কয় বছরে কত কী বদলেছে তুমি...,surprise,2
8,8,রঙে রঙে ছয়লাপতবু ভয়…এও একধরণের খেলা ভয়ের সঙ্গে...,fear,5
9,9,সময়টা আজ কেমন যেন বড় হয়ে গেছি আমি তারা গুলো...,sadness,1


In [ ]:
raw_test_data.head(10)

,Unnamed: 0,cleaned,classes,labels
0,0,দুই বছরের ব্যবধানে বগুড়া থেকে বিদেশে পণ্য রপ্ত...,joy,0
1,1,আওয়ামী লীগ সরকার বাংলাদেশকে অন্ধকার কুয়া থেকে ...,joy,0
2,2,চোখ কচলাতে কচলাতে বেশ রাগান্বীত কন্ঠ বলল ধুরো ...,disgust,3
3,3,মেয়েটা অতিরিক্ত আত্মপরিচয় সংকটে ভুগে সাবিলা স্...,disgust,3
4,4,এবার উন্নয়ন মুলক কাজের কিছু দৃশ্যমান পরিদশন ক...,joy,0
5,5,আওয়ামী লীগ সরকারের টানা ১০ বছর অর্থমন্ত্রীর দা...,disgust,3
6,6,স্কুইল নামক এক ঔষধি গাছের বিভিন্ন উপাদানের সাথ...,surprise,2
7,7,ভেবে খুব খারাপ লাগছে বাংলাদেশের মেধাবী মানুষগ...,sadness,1
8,8,তারপর হঠাৎ করেই আলেকজান্ডার তার বুকেফ্যালাসের ...,surprise,2
9,9,তো এই অবস্থায় আমি এসব সংবাদ আমি একদম নিতে পারি...,fear,5


In [ ]:

#Creating training data
train_data = raw_train_data.cleaned.tolist()
train_labels = raw_train_data.labels.tolist()

#Creating validation data
val_data = raw_val_data.cleaned.tolist()
val_labels = raw_val_data.labels.tolist()

#Creating testing data
test_data = raw_test_data.cleaned.tolist()
test_labels = raw_test_data.labels.tolist()


def sanity_print(x, y):
   print(len(x), len(y), type(x), type(y))
   for i in range(10):
     print(x[i], y[i])

sanity_print(train_data, train_labels)
sanity_print(val_data, val_labels)
sanity_print(test_data, test_labels)

4994 4994 <class 'list'> <class 'list'>
মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মনে হতো নিজেকেখাপছাড়া লাগতোদম বন্ধ হয়ে আসতোঅন্যদের সামনে অপমানিত হতে হতো পদে পদে 1
অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে  কেননা আমার সদ্য পাশ করা শিক্ষার্থীর মত এখন আমিও বেকার বেকার পরিচয় দিয়ে আম্মার কাছ থেকে আজ পঞ্চাশ টাকা ডাকাতি করলাম  1
আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা মুখার্জি কে এখন তেমনি সবচেয়ে বেশি ঘৃণা করি ওই পতিতা সদ্য হিন্দু হওয়া মিথিলা মুখার্জি কে 3
মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভুল তবে এই ভুলের মাশুল এইভাবেই কি দিতে হতো জানি না তবে যা জানি তা মানি না 1
এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের আর কেউ বেঁচে থাকবে না ও তোমাদের কাউকে বাঁচতে দিবে না  চলে যাও এখান থেকে নিজে থেকে অভিশাপকে আহ্বান করো না 5
কান্নার জল সবাই দেখে হৃদয়ের কষ্ট কেউ দেখেনা পাওয়ার আনন্দ কিছু দিন থাকেকিন্তু না পাওয়ার বেদনা সারাজীবন এ ও ভুলা যায়না 1
নির্বাচনের সময় কস্ট করে ভোট চাইসে এর চেয়ে বেশি এইসব প্রতিবন্ধী থেকে আর কি আশা করা যায় 3
দাদা আপনার video গুলো ভালো লাগে ডকুমেন্টারি টাইপের ভিডিও 

## ML Models

In [ ]:
%%time
lr_model = LogisticRegressionCV(class_weight='balanced', max_iter=400)
dt_model = DecisionTreeClassifier(class_weight='balanced')
rf_model = RandomForestClassifier(class_weight='balanced')
mnb_model = MultinomialNB()
svm_model = SVC(class_weight='balanced')
knn_model = KNeighborsClassifier()
ada_model = AdaBoostClassifier()
model_names = ['Logistic Regression','Decision Tree','Random Forest','Naive Bayes', 'SVM', 'KNN', 'AdaBoost']
ml_models = [lr_model,dt_model,rf_model,mnb_model, svm_model, knn_model, ada_model]

CPU times: user 162 µs, sys: 18 µs, total: 180 µs
Wall time: 183 µs


In [ ]:
%%time
def model_train(model, train_data, train_labels):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(train_data, train_labels)
  return pipe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,train_data, train_labels)
    trained_ml_models.append(tt)

### Performance on validation set

In [ ]:
pr_list = []
re_list = []
f1_list = []

def print_results(md, x_test, y_test, name):
    #print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
    # print(confusion_matrix(true,pred))
    # print(classification_report(true,pred))
    # print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
    # print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
    # print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, val_data, val_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score
Logistic Regression,66.10,65.71,65.84
Decision Tree,43.66,43.27,43.15
Random Forest,58.39,57.85,57.09
Naive Bayes,62.00,56.41,52.23
SVM,64.64,62.98,62.67
KNN,49.14,47.76,47.56
AdaBoost,49.67,44.07,43.06


### Performance on test set

In [ ]:
pr_list = []
re_list = []
f1_list = []

def print_results(md, x_test, y_test, name):
    #print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
    # print(confusion_matrix(true,pred))
    # print(classification_report(true,pred))
    # print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
    # print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
    # print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, test_data, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =model_names)
performance_matrix

,Precision,Recall,F1 Score
Logistic Regression,61.07,60.64,60.76
Decision Tree,44.51,44.16,44.23
Random Forest,57.95,56.48,54.85
Naive Bayes,64.24,54.08,48.68
SVM,61.13,60.00,59.54
KNN,44.55,44.64,44.07
AdaBoost,48.72,42.40,40.76


## Ensemble

In [ ]:
whole_data = []

def make_whole_list(L):
  for i in L:
    whole_data.append(i)

make_whole_list(train_data)
make_whole_list(val_data)
make_whole_list(test_data)
print(len(whole_data))

6243


In [ ]:
tf = TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(),  max_features=20000)
tf.fit_transform(whole_data) ### Bit confusion arises here need to resolve
trained_texts = tf.transform(train_data)
val_texts = tf.transform(val_data)
test_texts = tf.transform(test_data)

In [ ]:
%%time
from sklearn.ensemble import VotingClassifier
LogReg_clf = LogisticRegressionCV(class_weight='balanced', max_iter=400)
rt_clf =   RandomForestClassifier(class_weight='balanced')
SVC_clf = SVC(class_weight='balanced')


voting_clf = VotingClassifier(estimators=[('SVC', SVC_clf), ('LogReg', LogReg_clf),('RF', rt_clf)], voting='hard')
voting_clf.fit(trained_texts, train_labels)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


CPU times: user 2min 51s, sys: 2min 56s, total: 5min 47s
Wall time: 3min 5s


### Val set

In [ ]:
preds = voting_clf.predict(val_texts)
true, pred = val_labels, preds
print(confusion_matrix(true,pred))
print(classification_report(true,pred))
print("\nPrecison : ",precision_score(true, pred, average = 'weighted'))
print("\nRecall : ",recall_score(true, pred,  average = 'weighted'))
print("\nF1 : ",f1_score(true, pred,  average = 'weighted'))

[[ 77  21   5  14   2   1]
 [ 14  82   5  15   6   7]
 [  9  10  29   9   0   7]
 [  9   9   2 124  10   1]
 [  5  10   2  14  34   2]
 [  4  12   2  10   2  59]]
              precision    recall  f1-score   support

           0       0.65      0.64      0.65       120
           1       0.57      0.64      0.60       129
           2       0.64      0.45      0.53        64
           3       0.67      0.80      0.73       155
           4       0.63      0.51      0.56        67
           5       0.77      0.66      0.71        89

    accuracy                           0.65       624
   macro avg       0.65      0.62      0.63       624
weighted avg       0.65      0.65      0.65       624


Precison :  0.6517967728154874

Recall :  0.6490384615384616

F1 :  0.6455798649356003


### Test set

In [ ]:
preds = voting_clf.predict(test_texts)
true, pred = test_labels, preds
print(confusion_matrix(true,pred))
print(classification_report(true,pred))
print("\nPrecison : ",precision_score(true, pred, average = 'weighted'))
print("\nRecall : ",recall_score(true, pred,  average = 'weighted'))
print("\nF1 : ",f1_score(true, pred,  average = 'weighted'))

[[ 91   8   5  10   0   0]
 [ 15  65   2  22   7   8]
 [ 10  16  27   9   3   8]
 [ 12  18   4 115  14   2]
 [  7  12   2  25  23   2]
 [  5  12   3   8   1  54]]
              precision    recall  f1-score   support

           0       0.65      0.80      0.72       114
           1       0.50      0.55      0.52       119
           2       0.63      0.37      0.47        73
           3       0.61      0.70      0.65       165
           4       0.48      0.32      0.39        71
           5       0.73      0.65      0.69        83

    accuracy                           0.60       625
   macro avg       0.60      0.56      0.57       625
weighted avg       0.60      0.60      0.59       625


Precison :  0.598349179402831

Recall :  0.6

F1 :  0.590867371798098
